# Style Transfer model training, converting to ml5.js/tf.js and video stlye transfer

This notebook is based on the [Google Colab notebook](https://colab.research.google.com/drive/1TZNdhoUEBoxQqY5EFloZcuyYUs9oNQ-g), provided by ml5 and adapted so it works with Paperspace Gradient notebooks.  

It excludes downloading of the 14GB+ COCO image dataset, since it is already provided by Paperspace.  

First, we prepare the environment by downloading the fast-style-transfer git repo and the vgg image, installing libraries that are not included in the Paperspace TensorFlow 1.14 Docker image, train the model, convert it into ml5 appropriate format and (optionally) use it to perform stlye transfer on a video.

NOTE: For this notebook to work, the Paperspace Docker image must be TensorFlow 1.14.


## 1. Preparing the environment


In [ ]:
# Upgrade pip
pip install --upgrade pip

In [ ]:
# Clone the fast-style-transfer git repo from github: https://github.com/lengstrom/fast-style-transfer.
!git clone https://github.com/lengstrom/fast-style-transfer

Cloning into 'fast-style-transfer'...


remote: Enumerating objects: 248, done.
remote: Counting objects:  10% (1/10)
remote: Counting objects:  20% (2/10)
remote: Counting objects:  30% (3/10)
remote: Counting objects:  40% (4/10)
remote: Counting objects:  50% (5/10)
remote: Counting objects:  60% (6/10)
remote: Counting objects:  70% (7/10)
remote: Counting objects:  80% (8/10)
remote: Counting objects:  90% (9/10)
remote: Counting objects: 100% (10/10)
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects:  10% (1/10)
remote: Compressing objects:  20% (2/10)
remote: Compressing objects:  30% (3/10)
remote: Compressing objects:  40% (4/10)
remote: Compressing objects:  50% (5/10)
remote: Compressing objects:  60% (6/10)
remote: Compressing objects:  70% (7/10)
remote: Compressing objects:  80% (8/10)
remote: Compressing objects:  90% (9/10)
remote: Compressing objects: 100% (10/10)
remote: Compressing objects: 100% (10/10), done.


Receiving objects:   0% (1/248)   
Receiving objects:   1% (3/248)   
Receiving objects:   2% (5/248)   
Receiving objects:   3% (8/248)   
Receiving objects:   4% (10/248)   
Receiving objects:   5% (13/248)   
Receiving objects:   6% (15/248)   
Receiving objects:   7% (18/248)   
Receiving objects:   8% (20/248)   
Receiving objects:   9% (23/248)   
Receiving objects:  10% (25/248)   
Receiving objects:  11% (28/248)   
Receiving objects:  12% (30/248)   
Receiving objects:  13% (33/248)   
Receiving objects:  14% (35/248)   
Receiving objects:  15% (38/248)   
Receiving objects:  16% (40/248)   
Receiving objects:  17% (43/248)   
Receiving objects:  18% (45/248)   
Receiving objects:  19% (48/248)   
Receiving objects:  20% (50/248)   
Receiving objects:  21% (53/248)   
Receiving objects:  22% (55/248)   
Receiving objects:  23% (58/248)   
Receiving objects:  24% (60/248)   
Receiving objects:  25% (62/248)   
Receiving objects:  26% (65/248)   
Receiving objects:  27% (67/248)

Receiving objects:  37% (92/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  38% (95/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  39% (97/248), 7.37 MiB | 14.37 MiB/s   


Receiving objects:  41% (102/248), 7.37 MiB | 14.37 MiB/s   
remote: Total 248 (delta 2), reused 5 (delta 0), pack-reused 238
Receiving objects:  42% (105/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  43% (107/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  44% (110/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  45% (112/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  46% (115/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  47% (117/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  48% (120/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  49% (122/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  50% (124/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  51% (127/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  52% (129/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  53% (132/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  54% (134/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  55% (137/248), 7.37 MiB | 14.37 MiB/s   
Receiving objects:  

### Creating directories

Create a folder called 'ckpt' inside of 'fast-style-transfer' folder

In [ ]:
%mkdir /notebooks/fast-style-transfer/ckpt

Create another folder called 'images' inside of 'fast-style-transfer' folder

In [ ]:
%mkdir /notebooks/fast-style-transfer

Inside of the 'images' folder, create a folder called 'style'

In [ ]:
%mkdir /notebooks/fast-style-transfer/images/style

### Add a style image
- Go to left sidebar, click on the folder icon to open "Files" panel
- Put a style image inside of the _fast-style-transfer/images/style_ folder

### Install additional libraries

#### Install a higher version of tensorflow for training

We need the 2.1.0 version for training and video style transfer.  
The TensorFlow 1.14 version is needed only to convert the trained model to ml5, but we can only upgrade a version in a Docker image and not downgrade - this is why we start with the 1.14 image.

In [ ]:
# Install a higher version of tensorflow for training
!pip install tensorflow-gpu==2.1.0

#### Install necessary libraries

In [ ]:
# Install imageio
!pip install imageio

#### Install libraries for video style transfer (optional)

In [ ]:
# Install libraries for video style transfer
!pip install ffmpeg
!pip install moviepy==1.0.2
!pip install scipy
# Downgrading the image-ffmpeg prevent some pipe error
!pip install imageio-ffmpeg==0.2.0

## 2. Preparing the training data

#### Check that the COCO dataset exists on Paperspace

In [ ]:
# Check for the coco dataset in the paperspace files
%ls /datasets/coco/

annotations/  coco_train2014/  coco_val2014/


#### Downloading the vgg .mat file

In [ ]:
# Create the necessary directory
%mkdir /notebooks/fast-style-transfer/data/

In [ ]:
# Go into the appropriate directory and download the necessary file
%cd /notebooks/fast-style-transfer/data/
!wget https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat

/notebooks/fast-style-transfer/data


--2021-12-10 15:55:21--  https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
Resolving www.vlfeat.org (www.vlfeat.org)... 

64.90.48.57
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:443... 

connected.


HTTP request sent, awaiting response... 

...


                ima  99%[==================> ] 547.04M  3.97MB/s    eta 1s     


               imag  99%[==================> ] 547.74M  3.93MB/s    eta 0s     


              image  99%[==================> ] 548.55M  3.90MB/s    eta 0s     


             imagen  99%[==================> ] 549.10M  3.63MB/s    eta 0s     


imagenet-vgg-veryde 100%[===================>] 549.36M  3.64MB/s    in 2m 27s  

2021-12-10 15:57:48 (3.73 MB/s) - ‘imagenet-vgg-verydeep-19.mat’ saved [576042600/576042600]



In [ ]:
# Check that we got the .mat file
%ls /notebooks/fast-style-transfer/data

imagenet-vgg-verydeep-19.mat


## 3. Training with style.py

In [ ]:
# We first move into the fast-style-transfer repo
%cd /notebooks/fast-style-transfer

/notebooks/fast-style-transfer


Arguments - make sure 

- the --checkpoint-dir argument points to _/notebooks/fast-style-transfer/ckpt_
- the --style argument points to your style image, such as _/notebooks/fast-stype-transfer/images/style/slovenia-gan.jpg_
- the --vgg-path points to _/notebooks/fast-style-transfer/data/imagenet-vgg-verydeep-19.mat_


You can change the number after --epoch to change the training time, default to 2, it has to be > 0

Keep the below cell running, keep the tab active and wait. It took me 2 hours.

Paperspace might not update the cell output accordingly, so just be patient.

In [ ]:
# Remember to replace the name after 'images/style/' to your own style image name
!python style.py --checkpoint-dir /notebooks/fast-style-transfer/ckpt --style images/style/slovenia-gan.jpg --style-weight 1.5e2 --train-path --vgg-path /notebooks/fast-style-transfer/data/imagenet-vgg-verydeep-19.mat --epochs 2

At this point, you will be able to see 4 files in the '/fast-style-transfer/ckpt' folder.

## 4. Converting the model to ml5js

### Switch back to Tensorflow 1.14.0 

In [ ]:
!pip install tensorflow==1.14.0

### Download the repo for conversion

In [ ]:
# Let's go back to the root folder
%cd /notebooks

# Download the repo
!git clone https://github.com/reiinakano/fast-style-transfer-deeplearnjs.git

# Switch to the repo
%cd fast-style-transfer-deeplearnjs

/notebooks


Cloning into 'fast-style-transfer-deeplearnjs'...


remote: Enumerating objects: 1407, done.
Receiving objects:   0% (1/1407)   
Receiving objects:   1% (15/1407)   
Receiving objects:   2% (29/1407)   
Receiving objects:   3% (43/1407)   


Receiving objects:  16% (239/1407), 21.78 MiB | 21.77 MiB/s   


Receiving objects:  18% (254/1407), 21.78 MiB | 21.77 MiB/s   


Receiving objects:  27% (380/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  28% (394/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  29% (409/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  30% (423/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  31% (437/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  32% (451/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  33% (465/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  34% (479/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  35% (493/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  36% (507/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  37% (521/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  38% (535/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  39% (549/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  40% (563/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  41% (577/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  42% (591/1407), 32.45 MiB | 21.63 M

Receiving objects:  76% (1070/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  77% (1084/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  78% (1098/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  79% (1112/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  80% (1126/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  81% (1140/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  82% (1154/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  83% (1168/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  84% (1182/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  85% (1196/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  86% (1211/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  87% (1225/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  88% (1239/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  89% (1253/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  90% (1267/1407), 32.45 MiB | 21.63 MiB/s   
Receiving objects:  91% (1281/1407), 32.

/notebooks/fast-style-transfer-deeplearnjs


### Convert the model

Make sure that 

- the --output_dir argument points to your _ckpts/model_name_ (this directory will be created by this script)

in both script calls.

In [ ]:
# Change src/ckpts/slo-transfer to src/ckpts/YOUR_OWN_FOLDER
!python scripts/dump_checkpoint_vars.py --output_dir=src/ckpts/slo-transfer --checkpoint_file=/notebooks/fast-style-transfer/ckpt/fns.ckpt

# Change src/ckpts/slo-transfer to src/ckpts/YOUR_OWN_FOLDER
!python scripts/remove_optimizer_variables.py --output_dir=src/ckpts/slo-transfer

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

W1118 21:41:48.192328 139620861376320 deprecation_wrapper.py:119] From scripts/dump_checkpoint_vars.py:48: The name tf.train.NewCheckpointReader is deprecated. Please use tf.compat.v1.train.NewCheckpointReader instead.



W1118 21:41:48.200464 139620861376320 deprecation_wrapper.py:119] From scripts/dump_checkpoint_vars.py:51: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Writing variable Variable_9/Adam_1...
Writing variable Variable_8/Adam...
Writing variable Variable_8...
Writing variable Variable_7/Adam_1...
Writing variable Variable_7/Adam...
Writing variable Variable_7...
Writing variable Variable_6/Adam...
Writing variable Variable_6...
Writing variable Variable_5/Adam...
Writing variable Variable_47/Adam_1...
Writing variable Variable_45/Adam_1...
Writing variable Variable_45/Adam...
Writing variable Variable_45...
Writing variable Variable_43/Adam_1...
Writing variable Variable_43/Adam...
Writing variable Variable_43...
Writing variable Variable_5/Adam_1...
Writing variable Variable_42/Adam...


Writing variable Variable_41/Adam...
Writing variable Variable_41...
Writing variable Variable_40...
Writing variable Variable_39/Adam_1...
Writing variable Variable_39...
Writing variable Variable_38/Adam_1...
Writing variable Variable_4/Adam_1...
Writing variable Variable_38...
Writing variable Variable_44...
Writing variable Variable_20/Adam...
Writing variable Variable_10/Adam...
Writing variable Variable_37/Adam...
Writing variable Variable_31/Adam...
Writing variable Variable_19/Adam...
Writing variable Variable_25...
Writing variable Variable_9...
Writing variable Variable_32/Adam...
Writing variable Variable_34...


...

Writing variable Variable_23...
Writing variable Variable_10/Adam_1...
Writing variable Variable_24...
Writing variable Variable_37/Adam_1...
Writing variable Variable_47...
Writing variable Variable_14...
Writing variable Variable_20...
Writing variable Variable_30/Adam...
Writing variable Variable_1/Adam...
Writing variable Variable_29/Adam_1...
Writing variable Variable/Adam_1...
Writing variable Variable_19...
Writing variable Variable_21/Adam_1...
Writing variable Variable_11/Adam...
Writing variable Variable_29...
Writing variable Variable_22...
Writing variable Variable_16/Adam_1...
Writing variable beta1_power...
Writing variable Variable_40/Adam_1...
Writing variable Variable_1...
Writing variable Variable_44/Adam...
Writing variable Variable_12...


Writing variable Variable_36/Adam...
Writing variable Variable/Adam...
Writing variable Variable_25/Adam...
Writing variable Variable_12/Adam_1...
Writing variable Variable_23/Adam_1...
Writing variable Variable_12/Adam...
Writing variable Variable_21...
Writing variable Variable_27...
Writing variable Variable_4...
Writing variable Variable_37...
Writing variable Variable_10...
Writing variable Variable_13...
Writing variable Variable_28/Adam...
Writing variable Variable_33/Adam_1...
Writing variable Variable_13/Adam_1...


Writing variable Variable_14/Adam...
Writing variable Variable_3/Adam_1...
Writing variable Variable_18/Adam_1...
Writing variable Variable_2/Adam...
Writing variable Variable_31/Adam_1...
Writing variable Variable_9/Adam...
Writing variable Variable_42/Adam_1...
Writing variable Variable_22/Adam...
Writing variable Variable_6/Adam_1...
Writing variable Variable_22/Adam_1...
Writing variable Variable_23/Adam...
Writing variable Variable_28/Adam_1...
Writing variable Variable_35/Adam_1...
Writing variable Variable_20/Adam_1...
Writing variable Variable_26/Adam...
Writing variable Variable_40/Adam...
Writing variable Variable_4/Adam...
Writing variable Variable_30/Adam_1...
Writing variable Variable_33/Adam...
Writing variable Variable_26/Adam_1...
Writing variable Variable_26...
Writing variable Variable_27/Adam_1...
Writing variable Variable_1/Adam_1...
Writing variable Variable_29/Adam...
Writing variable Variable_25/Adam_1...
Writing variable Variable_28...
Writing variable Variable_

Writing variable Variable_13/Adam...
Writing variable Variable_27/Adam...
Writing variable Variable_33...
Writing variable Variable_34/Adam...
Writing variable Variable_34/Adam_1...
Writing variable Variable_46/Adam_1...
Writing variable Variable_24/Adam_1...
Writing variable Variable_35...
Writing variable Variable_35/Adam...
Writing variable Variable_46/Adam...
Writing variable Variable...
Writing variable Variable_36/Adam_1...
Writing manifest to src/ckpts/slo-gan/manifest.json


Done!


#### Zip the model and download it

Again, make sure the paths are correct for your model.

`tar cfz /notebooks/fast-style-transfer-deeplearnjs/src/ckpts/YOUR-MODEL-FOLDER.tar.gz /notebooks/fast-style-transfer-deeplearnjs/src/ckpts/YOUR-MODEL-FOLDER`

In [ ]:
# We use tar because no zip support
# Change /slo-transfer.tar.gz to /YOUR_FOLDER_NAME.tar.gz, same as src/ckpts/slo-transfer to /YOUR_FOLDER_NAME
!tar czf /notebooks/fast-style-transfer-deeplearnjs/src/ckpts/slo-transfer.tar.gz /notebooks/fast-style-transfer-deeplearnjs/src/ckpts/slo-transfer

tar: Removing leading `/' from member names


Now you should be able to see a YOUR_MODEL_NAME.tar.gz archived file from the Files panel (you might need to refresh the whole page if you don't see it there). Download it from the panel.

## 5. Run the model in ml5js

Put the downloaded and unzipped model folder into your [p5 sketch](https://editor.p5js.org/ml5/sketches/StyleTransfer_Image) in the _models/_ directory.

Change the model path in your p5 sketch:  
`style = ml5.styleTransfer('models/YOUR_MODEL_NAME', modelLoaded);`

## 6. Using the model for video style transfer

Make sure you uploaded the video you'd like to transfer the style to. You can upload it to the root folder (i.e. inside /notebooks)

Make sure you installed the required libraries before. You can also do so right here.

_Note: You do not need to convert the model to ml5 to use it for video style transfer here._

In [ ]:
# Install libraries for video style transfer
!pip install ffmpeg
!pip install moviepy==1.0.2
!pip install scipy
# Downgrading the image-ffmpeg prevent some pipe error
!pip install imageio-ffmpeg==0.2.0

We also need to install TensorFlow 2.10. version again. You can skip this step if you did not convert the model to ml5.

In [ ]:
!pip install tensorflow-gpu==2.1.0

#### Create the output-video directory

In [ ]:
%mkdir /notebooks/output-video

#### Style transfer!

Again - paths!

- the --in-path should point to your video, e.g. _/notebooks/MY_ORIGINAL_VIDEO.mp4_
- the --out-path should point to the _/notebooks/output-video/MY_TRANSFERED_VIDEO.mp4_

In [ ]:
# Go into the fast-style-transfer dir
%cd /notebooks/fast-style-transfer/

In [ ]:
# From here we run out process
# as stated here https://github.com/lengstrom/fast-style-transfer#stylizing-video

!python transform_video.py --in-path /notebooks/maribor.mp4 \
  --checkpoint ckpt/ \
  --tmp-dir /notebooks/tmp-out/ \
  --out-path /notebooks/output-video/video.mp4 \
  --device /gpu:0 \
  --batch-size 4

You should now see your style-transfered video in the output-video directory. Download it and enjoy!

***

Made by @zalashub on [GitHub](https://github.com/zalashub) and [Twitter](https://twitter.com/zalashub).